In [ ]:
import eli5
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import re
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.linear_model import Ridge
from math import sqrt
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('data/train.tsv', sep='\t', header=0)
dataset['category_name'] = dataset['category_name'].fillna('Other').astype(str)
dataset['brand_name'] = dataset['brand_name'].fillna('missing').astype(str)
dataset['shipping'] = dataset['shipping']
dataset['item_condition_id'] = dataset['item_condition_id']
dataset['item_description'] = dataset['item_description'].fillna('None')

In [3]:
dataset = dataset[dataset.price>0] # Remove entries with negative price

In [4]:
# split category into general, sub1, and sub2
categories = list(dataset['category_name'])

gencat_list = []
subcat1_list = []
subcat2_list = []

for cat in categories:
    cats = cat.split("/")
    cats[0] = cats[0].replace(' ','')
    cats[0] = cats[0].replace('&','_')
    if len(cats) > 1: 
        cats[1] = cats[1].replace(' ','')
        cats[1] = cats[1].replace('&','_')
    if len(cats) > 2: 
        cats[2] = cats[2].replace(' ','')
        cats[2] = cats[2].replace('&','_')
    gencat_list.append(cats[0].strip())
    subcat1_list.append(cats[1].strip() if len(cats) > 1 else "Other")
    subcat2_list.append(cats[2].strip() if len(cats) > 2 else "Other")

dataset["general_category"] = gencat_list
dataset["subcategory_1"] = subcat1_list
dataset["subcategory_2"] = subcat2_list
dataset.drop('category_name', axis=1, inplace=True)

In [5]:
X = dataset.loc[:, dataset.columns != 'price']
Y = np.log1p(dataset['price'])

# 80% training data, 20% test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [48]:
# one hot encode categorical features that are not already numbers

def one_hot_categorical(train, test):
    vectorizer = CountVectorizer(lowercase=False, binary=True)
    onehot_train = vectorizer.fit_transform(train.values)
    onehot_test = vectorizer.transform(test.values)
    return onehot_train, onehot_test

onehot_train_brand, onehot_test_brand = one_hot_categorical(X_train['brand_name'], X_test['brand_name'])
onehot_train_gencat, onehot_test_gencat = one_hot_categorical(X_train['general_category'], X_test['general_category'])
onehot_train_subcat1, onehot_test_subcat1 = one_hot_categorical(X_train['subcategory_1'], X_test['subcategory_1'])
onehot_train_subcat2, onehot_test_subcat2 = one_hot_categorical(X_train['subcategory_2'], X_test['subcategory_2'])

In [49]:
# Use tfidf for text-based features

def tfidf_text(train, test, min_df, max_features):
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=min_df, max_features=max_features)
    tfidf_train = vectorizer.fit_transform(train.values)
    tfidf_test = vectorizer.transform(test.values)
    return tfidf_train, tfidf_test
    
tfidf_train_name, tfidf_test_name = tfidf_text(X_train['name'], X_test['name'], 3, 250000)
tfidf_train_description, tfidf_test_description = tfidf_text(X_train['item_description'], X_test['item_description'], 5, 500000)

In [8]:
# Storing categorical features that are already numbers to sparse matrix

X_train_cat = csr_matrix(pd.get_dummies(X_train[['item_condition_id', 'shipping']], sparse=True).values)

X_test_cat = csr_matrix(pd.get_dummies(X_test[['item_condition_id', 'shipping']], sparse=True).values)

In [22]:
print(X_train_cat.shape, X_test_cat.shape)

(1185328, 2) (296333, 2)


In [50]:
# stack all categorical and text sparse matrices
train_x = hstack((onehot_train_brand, onehot_train_gencat, onehot_train_subcat1, onehot_train_subcat2, 
                       tfidf_train_name, tfidf_train_description, X_train_cat)).tocsr()
test_x = hstack((onehot_test_brand, onehot_test_gencat, onehot_test_subcat1, onehot_test_subcat2, 
                       tfidf_test_name, tfidf_test_description, X_test_cat)).tocsr()

In [51]:
print(train_x.shape, test_x.shape)

(1185328, 755935) (296333, 755935)


In [58]:
alpha = [1, 2, 3, 3.5, 4, 4.5, 5, 6, 7] 
test_rmsle_array=[] 
for i in tqdm(alpha):
    model = Ridge(solver="sag", random_state=42, alpha=i)
    model.fit(train_x, Y_train)
    preds_test = model.predict(test_x)
    test_rmsle_array.append(sqrt(mse(Y_test, preds_test)))

for i in range(len(test_rmsle_array)):
    print ('RMSLE for alpha = ',alpha[i],'is',test_rmsle_array[i])
    
best_alpha = np.argmin(test_rmsle_array)

fig, ax = plt.subplots()
ax.plot(alpha, test_rmsle_array)
ax.scatter(alpha, test_rmsle_array)
for i, txt in enumerate(np.round(test_rmsle_array,3)):
    ax.annotate((alpha[i],np.round(txt,3)), (alpha[i],test_rmsle_array[i]))

plt.title("Cross Validation Error for each alpha")
plt.xlabel("Alpha")
plt.ylabel("Error")
plt.show()

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]F:\Desktop\Classes\cs4120-hw4\.venv\lib\site-packages\sklearn\linear_model\_ridge.py:830: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  warnings.warn(
 11%|█████████▏                                                                         | 1/9 [02:11<17:32, 131.50s/it]F:\Desktop\Classes\cs4120-hw4\.venv\lib\site-packages\sklearn\linear_model\_ridge.py:830: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  warnings.warn(
 22%|██████████████████▍                                                                | 2/9 [03:

RMSLE for alpha =  1 is 0.443143070567117
RMSLE for alpha =  2 is 0.43805985358439264
RMSLE for alpha =  3 is 0.437508664007679
RMSLE for alpha =  3.5 is 0.4377504452105116
RMSLE for alpha =  4 is 0.43815227171846255
RMSLE for alpha =  4.5 is 0.4386530251699621
RMSLE for alpha =  5 is 0.4392109054475655
RMSLE for alpha =  6 is 0.44041454744644054
RMSLE for alpha =  7 is 0.44165225658510726


NameError: name 'plt' is not defined

In [61]:
print("Best alpha: ",  alpha[best_alpha])
model = Ridge(solver="sag", random_state=42, alpha=alpha[best_alpha])
model.fit(train_x, Y_train)
ridge_preds_train = model.predict(train_x)
ridge_preds_test = model.predict(test_x)

print('Train RMSLE:', sqrt(mse(Y_train, ridge_preds_train)))

ridge_rmsle = sqrt(mse(Y_test, ridge_preds_test))
print("Cross validation RMSLE: ", ridge_rmsle)

Best alpha:  3


F:\Desktop\Classes\cs4120-hw4\.venv\lib\site-packages\sklearn\linear_model\_ridge.py:830: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  warnings.warn(


Train RMSLE: 0.37399684032095765
Cross validation RMSLE:  0.437508664007679


In [18]:
X_train

,train_id,name,item_condition_id,brand_name,shipping,item_description,general_category,subcategory_1,subcategory_2
901879,901879,Black Nike Cortez Size 4,2,Nike,1,Size 4. My baby wore once before she was reall...,Kids,Boys2T-5T,Shoes
984774,984774,HP Pavilion DV5000 DV6000 DV8000 DV9000,3,HP,1,HP Pavilion DV5000 DV6000 DV8000 DV9000 Ac Ada...,Other,Other,Other
1059057,1059057,New 24pc Cake Decorating Kit,1,missing,1,New Cake Decorating Set - With 24 pcs Icing Ti...,Home,Kitchen_Dining,Bakeware
122771,122771,Marc by marc jacobs,2,MARC BY MARC JACOBS,0,Authentic light brown pebble leather marc by m...,Women,Women'sAccessories,Wallets
293900,293900,Samsung s7 edge brand new,1,Samsung,1,"Brand new still in box, everything included. C...",Electronics,CellPhones_Accessories,CellPhones_Smartphones
...,...,...,...,...,...,...,...,...,...
359953,359953,"2 Jacket, 1 Shirt And Bottoms",3,missing,0,"USPA, IZOD, PERRY ELLIS, LEVI",Other,Other,Other
152400,152400,White maxi dress perfect for summer!,2,missing,0,Never been worn! So beautiful for summer night...,Women,Skirts,Maxi
963931,963931,Barcelona Messi Soccer Jersey 2018 (M),1,missing,1,Barcelona Messi Soccer Jersey Size: Medium 100...,Men,AthleticApparel,Jerseys
118024,118024,32 pcs Vander makeup brushes,1,missing,1,Package includes: 14 x The Specifications Eyes...,Beauty,Tools_Accessories,MakeupBrushes_Tools
